In [4]:
ls

gdrive/  sample_data/


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [128]:
import os, re, glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image


groups_folder_path = "/content/gdrive/MyDrive/image set"
categories = ["front_logo", "gs25", "gusia", "insa_kiosk", "job", "lib_kiosk", "lib_science", "post_office"]
nb_classes = len(categories)

image_w = 64
image_h = 64

pixels = image_h * image_w * 3



X = []
y = []

for idx, category in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = groups_folder_path + "/" + category
    files = glob.glob(image_dir+"/*.jpeg")
    print(category, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 27 == 0:
            print(category, " : ", f)



front_logo  파일 길이 :  34
front_logo  :  /content/gdrive/MyDrive/image set/front_logo/IMG_5034.jpeg
front_logo  :  /content/gdrive/MyDrive/image set/front_logo/IMG_5058.jpeg
gs25  파일 길이 :  47
gs25  :  /content/gdrive/MyDrive/image set/gs25/IMG_5099.jpeg
gs25  :  /content/gdrive/MyDrive/image set/gs25/IMG_5125.jpeg
gusia  파일 길이 :  36
gusia  :  /content/gdrive/MyDrive/image set/gusia/IMG_5239.jpeg
gusia  :  /content/gdrive/MyDrive/image set/gusia/IMG_5262.jpeg
insa_kiosk  파일 길이 :  28
insa_kiosk  :  /content/gdrive/MyDrive/image set/insa_kiosk/IMG_5096.jpeg
insa_kiosk  :  /content/gdrive/MyDrive/image set/insa_kiosk/IMG_5095.jpeg
job  파일 길이 :  40
job  :  /content/gdrive/MyDrive/image set/job/IMG_5317.jpeg
job  :  /content/gdrive/MyDrive/image set/job/IMG_5347.jpeg
lib_kiosk  파일 길이 :  40
lib_kiosk  :  /content/gdrive/MyDrive/image set/lib_kiosk/IMG_5198.jpeg
lib_kiosk  :  /content/gdrive/MyDrive/image set/lib_kiosk/IMG_5217.jpeg
lib_science  파일 길이 :  46
lib_science  :  /content/gdrive/MyDriv

In [129]:
import numpy as np
X = np.array(X)
print("x done")
y = np.array(y)
#1 0 0 0 이면 airplanes
#0 1 0 0 이면 buddha 이런식

X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("./img_data.npy", xy)

print("ok", len(y))


x done
ok 318


/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [130]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
#import keras.backend.tensorflow_backend as K

import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
#session = tf.Session(config=config)

X_train, X_test, y_train, y_test = np.load('./img_data.npy',allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

(238, 64, 64, 3)
238


In [131]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense
from keras.layers import Flatten, Convolution2D, MaxPooling2D
from keras.models import load_model
import cv2
from tensorflow.keras.optimizers import Adam
#from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

X_train, X_test, y_train, y_test = np.load('./img_data.npy',allow_pickle=True)

model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dir = './model'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + '/multi_img_classification.model'
checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [132]:
model.summary()

Model: "sequential_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_166 (Conv2D)         (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_95 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 dropout_132 (Dropout)       (None, 32, 32, 32)        0         
                                                                 
 conv2d_167 (Conv2D)         (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_96 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 dropout_133 (Dropout)       (None, 16, 16, 64)      

In [137]:
history = model.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

Epoch 1/30
7/8 [=========================>....] - ETA: 0s - loss: 0.0341 - accuracy: 0.9911
Epoch 1: val_loss did not improve from 0.23384
8/8 [==============================] - 1s 71ms/step - loss: 0.0397 - accuracy: 0.9874 - val_loss: 0.3135 - val_accuracy: 0.9250
Epoch 2/30
7/8 [=========================>....] - ETA: 0s - loss: 0.1636 - accuracy: 0.9598
Epoch 2: val_loss did not improve from 0.23384
8/8 [==============================] - 0s 44ms/step - loss: 0.1555 - accuracy: 0.9622 - val_loss: 0.5239 - val_accuracy: 0.9500
Epoch 3/30
7/8 [=========================>....] - ETA: 0s - loss: 0.1413 - accuracy: 0.9509
Epoch 3: val_loss did not improve from 0.23384
8/8 [==============================] - 0s 44ms/step - loss: 0.1505 - accuracy: 0.9496 - val_loss: 0.5178 - val_accuracy: 0.9125
Epoch 4/30
7/8 [=========================>....] - ETA: 0s - loss: 0.0313 - accuracy: 0.9821
Epoch 4: val_loss improved from 0.23384 to 0.22478, saving model to ./model/multi_img_classification.model


In [140]:
model.save('dicswunary.h5')

In [138]:
print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))

3/3 [==============================] - 0s 10ms/step - loss: 0.2740 - accuracy: 0.9750
정확도 : 0.9750


In [139]:
from PIL import Image
import os, glob, numpy as np
from keras.models import load_model

caltech_dir = "./guide/"
image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)
model = load_model('./model/multi_img_classification.model')

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0

#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    print(i)
    # print(pre_ans)
    pre_ans_str = ''
    category = ["front_logo", "gs25", "gusia", "insa_kiosk", "job", "lib_kiosk", "lib_science", "post_office"]
    if pre_ans == 0: pre_ans_str = category[0]
    elif pre_ans == 1: pre_ans_str = category[1]
    elif pre_ans == 2: pre_ans_str = category[2]
    elif pre_ans == 3: pre_ans_str = category[3]
    elif pre_ans == 4: pre_ans_str = category[4]
    elif pre_ans == 5: pre_ans_str = category[5]
    elif pre_ans == 6: pre_ans_str = category[6]
    else : pre_ans_str = category[7]

    
    if i[0] >= 0.8: print("해당 "+filenames[cnt]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[1] >= 0.8: print("해당 "+filenames[cnt]+"이미지는 "+pre_ans_str+"으로 추정됩니다.")
    if i[2] >= 0.8: print("해당 "+filenames[cnt]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[3] >= 0.8: print("해당 "+filenames[cnt]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[4] >= 0.8: print("해당 "+filenames[cnt]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[5] >= 0.7: print("해당 "+filenames[cnt]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[6] >= 0.8: print("해당 "+filenames[cnt]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[7] >= 0.7: print("해당 "+filenames[cnt]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    
    cnt += 1

[0.000 0.000 0.001 0.000 0.002 0.000 0.000 0.997]
해당 ./guide/IMG_5298.jpeg이미지는 post_office로 추정됩니다.
[0.995 0.000 0.003 0.002 0.000 0.000 0.000 0.000]
해당 ./guide/IMG_5047.jpeg이미지는 front_logo로 추정됩니다.
[0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000]
해당 ./guide/IMG_5101.jpeg이미지는 gs25으로 추정됩니다.
[0.000 0.000 0.000 0.033 0.938 0.001 0.000 0.029]
해당 ./guide/IMG_5650.jpeg이미지는 job로 추정됩니다.
[0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000]
해당 ./guide/IMG_5188.jpeg이미지는 lib_science로 추정됩니다.
[0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000]
해당 ./guide/IMG_5099.jpeg이미지는 gs25으로 추정됩니다.
[0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000]
해당 ./guide/IMG_5340.jpeg이미지는 job로 추정됩니다.
[0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000]
해당 ./guide/IMG_5654.jpeg이미지는 insa_kiosk로 추정됩니다.
[0.126 0.124 0.128 0.117 0.119 0.130 0.123 0.134]
[0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000]
해당 ./guide/IMG_5252.jpeg이미지는 gusia로 추정됩니다.


In [136]:
i

array([0.005, 0.008, 0.736, 0.005, 0.214, 0.014, 0.010, 0.010],
      dtype=float32)

In [102]:
prediction

array([[0.000, 1.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000],
       [0.000, 0.000, 0.000, 0.002, 0.006, 0.989, 0.003, 0.000],
       [0.000, 1.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000],
       [0.018, 0.001, 0.000, 0.000, 0.240, 0.741, 0.000, 0.000],
       [0.000, 0.000, 0.000, 1.000, 0.000, 0.000, 0.000, 0.000],
       [0.515, 0.036, 0.100, 0.028, 0.042, 0.063, 0.108, 0.107],
       [0.383, 0.107, 0.000, 0.000, 0.003, 0.095, 0.012, 0.399]],
      dtype=float32)

Epoch 1/20
15/15 [==============================] - 1s 41ms/step - loss: 74.1381 - accuracy: 0.3403
Epoch 2/20
15/15 [==============================] - 1s 40ms/step - loss: 0.1880 - accuracy: 0.9412
Epoch 3/20
15/15 [==============================] - 1s 41ms/step - loss: 0.0330 - accuracy: 0.9874
Epoch 4/20
15/15 [==============================] - 1s 42ms/step - loss: 0.0147 - accuracy: 0.9958
Epoch 5/20
15/15 [==============================] - 1s 44ms/step - loss: 0.0209 - accuracy: 0.9958
Epoch 6/20
15/15 [==============================] - 1s 43ms/step - loss: 0.0070 - accuracy: 0.9958
Epoch 7/20
15/15 [==============================] - 1s 45ms/step - loss: 0.0140 - accuracy: 0.9958
Epoch 8/20
15/15 [==============================] - 1s 41ms/step - loss: 0.0140 - accuracy: 1.0000
Epoch 9/20
15/15 [==============================] - 1s 41ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 10/20
15/15 [==============================] - 1s 41ms/step - loss: 0.0217 - accuracy: 1.0000
Epoch 11

In [142]:
pip install coremltools

     |████████████████████████████████| 1.6 MB 5.4 MB/s 


In [155]:
import coremltools
coreml_model = coremltools.converters.convert('./dicswunary.h5')
coreml_model.save('./dicswunary.mlmodel')

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 45/45 [00:00<00:00, 109.15 ops/s]
